In [1]:
from cluster_scripts import *

# This imports all the important definitions from

/home/shixubenjamin/anaconda3/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


### Table S2 - O<sub>2</sub> binding energy of various electronic structure methods

In [2]:
# Table of O2 binding energy. We also use this cell to calculate the O_atom energies which will be required for calculating EOv later on.

# Initializing total energy of O atom for PBE at def2-SVP + def2-TZVPP basis sets and PBE0 at def2-TZVPP level
O_ene_PBE_SVP = find_energy('02-Simulation_Data/Table_S2/Calculations/O/PBE_SVP/mrcc.out',typ='dft')
O_ene_PBE_TZVPP = find_energy('02-Simulation_Data/Table_S2/Calculations/O/PBE_TZVPP/mrcc.out',typ='dft')
O_ene_PBE0_TZVPP = find_energy('02-Simulation_Data/Table_S2/Calculations/O/PBE0_TZVPP/mrcc.out',typ='dft')
O_ene_HF_TZVPP = find_energy('02-Simulation_Data/Table_S2/Calculations/O/LMP2_TZ/mrcc.out',typ='hf')
O_ene_LMP2_TZVPP = O_ene_HF_TZVPP + find_energy('02-Simulation_Data/Table_S2/Calculations/O/LMP2_TZ/mrcc.out',typ='mp2')
O_ene_CCSDT_TZVPP = O_ene_HF_TZVPP + find_energy('02-Simulation_Data/Table_S2/Calculations/O/CCSDT_TZ/mrcc.out',typ='ccsdt')

# Binding energies will be saved in the dictionary: O2_bind_ene
O2_bind_ene = {}

# Starting with DFT (excluding double-hybrids) - all performed at def2-QZVPP level
dft_xc_functionals = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X']

for i in dft_xc_functionals:
        ene_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}/O2/mrcc.out'.format(i),typ='dft')
        ene_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}/O_O1/mrcc.out'.format(i),typ='dft')
        ene_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}/O_O2/mrcc.out'.format(i),typ='dft')
        O2_bind_ene[i] = (ene_O_O1 + ene_O_O2 - ene_O2)*Hartree


# Now for cWFT methods - we perform a def2-TZVPP/QZVPP basis set extrapolation here.

ene_bind_hf = []
ene_bind_ccsd = []
ene_bind_mp2 = []
ene_bind_ccsdt = []

dh_functional_list = ['B2PLYP']

for i in dh_functional_list:

    ene_bind_hf = []
    ene_bind_mp2 = []
    for j in ['TZ','QZ']:
        ene_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O2/mrcc.out'.format(i,j),typ='hf')
        ene_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O_O1/mrcc.out'.format(i,j),typ='hf')
        ene_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O_O2/mrcc.out'.format(i,j),typ='hf')
        ene_bind_hf += [(ene_O_O1 + ene_O_O2 - ene_O2)*Hartree]
        ene_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O2/mrcc.out'.format(i,j),typ='{0}'.format(i))
        ene_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O_O1/mrcc.out'.format(i,j),typ='{0}'.format(i))
        ene_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/{0}_{1}/O_O2/mrcc.out'.format(i,j),typ='{0}'.format(i))
        ene_bind_mp2 += [(ene_O_O1 + ene_O_O2 - ene_O2)*Hartree]
    
    O2_bind_ene['L{0}'.format(i)] = extrapolate.get_cbs(ene_bind_hf[0],ene_bind_mp2[0],ene_bind_hf[1],ene_bind_mp2[1],X=3,Y=4,family='def2',output=False)[2]

ene_bind_hf = []
ene_bind_ccsd = []
ene_bind_mp2 = []
ene_bind_ccsdt = []

for i in ['TZ','QZ']:
    ene_hf_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O2/mrcc.out'.format(i),typ='hf')
    ene_hf_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O1/mrcc.out'.format(i),typ='hf')
    ene_hf_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O2/mrcc.out'.format(i),typ='hf')
    ene_bind_hf += [(ene_hf_O_O1 + ene_hf_O_O2 - ene_hf_O2)*Hartree]
    ene_ccsdt_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O2/mrcc.out'.format(i),typ='ccsdt')
    ene_ccsdt_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O1/mrcc.out'.format(i),typ='ccsdt')
    ene_ccsdt_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O2/mrcc.out'.format(i),typ='ccsdt')
    ene_mp2_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O2/mrcc.out'.format(i),typ='mp2')
    ene_mp2_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O1/mrcc.out'.format(i),typ='mp2')
    ene_mp2_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O2/mrcc.out'.format(i),typ='mp2')
    ene_ccsd_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O2/mrcc.out'.format(i),typ='ccsd')
    ene_ccsd_O_O1 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O1/mrcc.out'.format(i),typ='ccsd')
    ene_ccsd_O_O2 = find_energy('02-Simulation_Data/Table_S2/Calculations/O2/CCSDT_{0}/O_O2/mrcc.out'.format(i),typ='ccsd')
    ene_bind_ccsdt += [(ene_ccsdt_O_O1 + ene_ccsdt_O_O2 - ene_ccsdt_O2)*Hartree]
    ene_bind_mp2 += [(ene_mp2_O_O1 + ene_mp2_O_O2 - ene_mp2_O2)*Hartree]
    ene_bind_ccsd += [(ene_ccsd_O_O1 + ene_ccsd_O_O2 - ene_ccsd_O2)*Hartree]
    print(ene_bind_hf,ene_bind_ccsd)

O2_bind_ene['LMP2'] = extrapolate.get_cbs(ene_bind_hf[0],ene_bind_mp2[0],ene_bind_hf[1],ene_bind_mp2[1],X=3,Y=4,family='def2',output=False)[2]
O2_bind_ene['LNO-CCSD'] = extrapolate.get_cbs(ene_bind_hf[0],ene_bind_ccsd[0],ene_bind_hf[1],ene_bind_ccsd[1],X=3,Y=4,family='def2',output=False)[2]
O2_bind_ene['LNO-CCSD(T)'] = extrapolate.get_cbs(ene_bind_hf[0],ene_bind_ccsdt[0],ene_bind_hf[1],ene_bind_ccsdt[1],X=3,Y=4,family='def2',output=False)[2]

print('{0:15s}  |{1}'.format('Method','Ebind (eV)'))
print('---------------------------')
for method,eov in O2_bind_ene.items():
    print('{0:15s}  |    {1:.3f}'.format(method, eov))


[1.4084845127484815] [3.1244020606705205]
[1.4084845127484815, 1.428198015915978] [3.1244020606705205, 3.265250419372299]
Method           |Ebind (eV)
---------------------------
PBE              |    6.215
BP86             |    6.146
BLYP             |    5.858
TPSS             |    5.486
SCAN             |    5.516
PBE0             |    5.379
HSE06            |    5.329
B3LYP            |    5.321
wB97X            |    5.407
LB2PLYP          |    5.359
LMP2             |    5.651
LNO-CCSD         |    4.800
LNO-CCSD(T)      |    5.163


### Table S8 - Bulk limit EOv values from periodic supercell approach

In [3]:
# Get total energy of O atom for PBE and PBE0 XC functionals

O_ene_PBE_VASP = np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_O_VASP')[0,1]
O_ene_PBE0_VASP = np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_O_VASP')[1,1]

eov_bulk_lim = {}

for index,i in enumerate(list(np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_metal-oxide_VASP',dtype=str)[:,0])):
    if 'PBE0' in i:
        eov_bulk_lim[i] = np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_metal-oxide_VASP',dtype=float)[index,2] + O_ene_PBE0_VASP \
            - np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_metal-oxide_VASP')[index,1]  - O2_bind_ene['PBE0']/2
    elif 'PBE' in i:
        eov_bulk_lim[i] = np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_metal-oxide_VASP',dtype=float)[index,2] + O_ene_PBE_VASP \
            - np.genfromtxt('02-Simulation_Data/Table_S8-Bulk_Limit_EOv/energies_metal-oxide_VASP')[index,1] - O2_bind_ene['PBE']/2

# Adding PBE0_2x6 by using PBE finite size corrections

eov_bulk_lim['PBE0_TiO2_Surface_2x6'] = eov_bulk_lim['PBE_TiO2_Surface_2x6'] - eov_bulk_lim['PBE_TiO2_Surface_2x4'] + eov_bulk_lim['PBE0_TiO2_Surface_2x4']

print('{0:30s}  |{1}'.format('System','EOv (eV)'))
print('-'*45)
for system,eov in eov_bulk_lim.items():
    print('{0:30s}  |   {1:.3f}'.format(system, eov))

System                          |EOv (eV)
---------------------------------------------
PBE_MgO_Bulk                    |   6.647
PBE_MgO_Surface                 |   6.123
PBE_TiO2_Bulk                   |   5.614
PBE_TiO2_Surface_2x4            |   5.101
PBE_TiO2_Surface_2x6            |   5.075
PBE0_TiO2_Surface_2x4           |   5.470
PBE0_TiO2_Surface_2x6           |   5.445


### Figure 1 - Schematic graph comparing RDF and Inconsistent approaches

In [4]:
# Plots out the schematic graph in Fig_01

fig, axs = plt.subplots(1,3,figsize=(6.69,3),dpi=1200)

for ax in axs[:2]:
    ax.remove()

RDF_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,0]
RDF_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025
RDF_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025

SVP2TZVPP_corr = np.average(RDF_eov[9:] - RDF_eov_SVP[9:]) # This shifts the SVP calculation to TZVPP level for the stoichoimetric clusters

Stoic_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,0]
Stoic_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 -0.025 + SVP2TZVPP_corr 

axs[2].plot([-2,130], np.array([eov_bulk_lim['PBE_TiO2_Bulk'],eov_bulk_lim['PBE_TiO2_Bulk']]),'k--',linewidth=1.5,label='Bulk limit')

axs[2].plot(np.append(RDF_sizes[::2],89),np.append(RDF_eov[::2],RDF_eov[-1]), label='RDF approach')
axs[2].plot(Stoic_sizes,Stoic_eov, label=r'Inconsistent approach')

axs[2].set_ylabel(r'O vacancy formation energy $\left ( E_\textrm{Ov} \right )$')
axs[2].set_xlabel(r'Quantum cluster size')
axs[2].set_xlim([0,92])
axs[2].set_ylim([5.2,6.3])
axs[2].legend(fancybox=False,fontsize=8,frameon=False,loc='upper right')
axs[2].set_yticklabels([])
axs[2].set_xticklabels([])
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)

plt.tight_layout(pad=3)
plt.savefig('03-Figures/Fig_01.eps')



In [17]:
# Plots out the schematic graph in Fig_01

fig, axs = plt.subplots(1,3,figsize=(6.69,3),dpi=1200)

for ax in axs[:2]:
    ax.remove()

RDF_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,0]
RDF_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025
RDF_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025

SVP2TZVPP_corr = np.average(RDF_eov[9:] - RDF_eov_SVP[9:]) # This shifts the SVP calculation to TZVPP level for the stoichoimetric clusters

Stoic_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,0]
Stoic_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 -0.025 + SVP2TZVPP_corr 




axs[2].plot([-2,130], np.array([eov_bulk_lim['PBE_TiO2_Bulk'],eov_bulk_lim['PBE_TiO2_Bulk']]),'k--',linewidth=1.5,label='Bulk limit')

axs[2].fill_between([-10,150],[eov_bulk_lim['PBE_TiO2_Bulk']+0.05, eov_bulk_lim['PBE_TiO2_Bulk']+0.05],[eov_bulk_lim['PBE_TiO2_Bulk']-0.05,\
     eov_bulk_lim['PBE_TiO2_Bulk']-0.05],color='tab:gray',edgecolor=None,alpha=0.2,label='Acceptable error')

axs[2].plot(Stoic_sizes,Stoic_eov, color='r',label=r'Inconsistent approach')

# axs[2].plot(np.append(RDF_sizes[::2],89),np.append(RDF_eov[::2],RDF_eov[-1]), label='RDF approach')

axs[2].set_ylabel(r'O vacancy formation energy $\left ( E_\textrm{Ov} \right )$')
axs[2].set_xlabel(r'Quantum cluster size')
axs[2].set_xlim([0,92])
axs[2].set_ylim([5.2,6.3])
axs[2].legend(fancybox=False,fontsize=7,frameon=False,loc='upper right')
axs[2].set_yticklabels([])
axs[2].set_xticklabels([])
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)

plt.tight_layout(pad=3)
plt.savefig('03-Figures/Fig_01_RIG_talk_01.png')



In [18]:
Stoic_eov

array([5.18000395, 5.73813031, 5.71596392, 5.32947351, 5.45409401,
       5.29970469, 5.42284802, 5.59247605, 5.46438619, 5.51010162,
       5.6034294 ])

In [16]:
# Plots out the schematic graph in Fig_01

fig, axs = plt.subplots(1,3,figsize=(6.69,3),dpi=1200)

for ax in axs[:2]:
    ax.remove()

RDF_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,0]
RDF_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025
RDF_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/RDF_SVP/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 - 0.025

SVP2TZVPP_corr = np.average(RDF_eov[9:] - RDF_eov_SVP[9:]) # This shifts the SVP calculation to TZVPP level for the stoichoimetric clusters

Stoic_sizes = np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,0]
Stoic_eov = (np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_01/Calculations/Stoic/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2 -0.025 + SVP2TZVPP_corr 

axs[2].plot([-2,130], np.array([eov_bulk_lim['PBE_TiO2_Bulk'],eov_bulk_lim['PBE_TiO2_Bulk']]),'k--',linewidth=1.5,label='Bulk limit')
axs[2].fill_between([-10,150],[eov_bulk_lim['PBE_TiO2_Bulk']+0.05, eov_bulk_lim['PBE_TiO2_Bulk']+0.05],[eov_bulk_lim['PBE_TiO2_Bulk']-0.05,\
     eov_bulk_lim['PBE_TiO2_Bulk']-0.05],color='tab:gray',edgecolor=None,alpha=0.2,label='Acceptable error')

axs[2].plot(Stoic_sizes,Stoic_eov, color='r',label=r'Inconsistent approach')

axs[2].plot(np.append(RDF_sizes[::2],89),np.append(RDF_eov[::2],RDF_eov[-1]), color='b',label='Systematic approach')

axs[2].set_ylabel(r'O vacancy formation energy $\left ( E_\textrm{Ov} \right )$')
axs[2].set_xlabel(r'Quantum cluster size')
axs[2].set_xlim([0,92])
axs[2].set_ylim([5.2,6.3])
axs[2].legend(fancybox=False,fontsize=7,frameon=False,loc='upper right')
axs[2].set_yticklabels([])
axs[2].set_xticklabels([])
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].spines['right'].set_visible(False)
axs[2].spines['top'].set_visible(False)

plt.tight_layout(pad=3)
plt.savefig('03-Figures/Fig_01_RIG_talk_02.png')



### Figure 2 - RDF versus Stoichiometric quantum clusters 

In [ ]:

RDF_sizes = np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_perfect_TZVPP')[:,0]
RDF_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

Stoic_sizes = np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_perfect_TZVPP')[:,0]
Stoic_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

Ti3_configurations_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_Ti3_configurations_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_Ti3_configurations_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

SCC_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/SCC/energies_defect_TZVPP')[1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/SCC/energies_perfect_TZVPP')[1])*Hartree - O2_bind_ene['PBE']/2

fig, axs = plt.subplots(figsize=(2.23,2.5),dpi=1200, constrained_layout=True)

axs.fill_between([-10,150],[eov_bulk_lim['PBE_TiO2_Bulk']+0.05, eov_bulk_lim['PBE_TiO2_Bulk']+0.05],[eov_bulk_lim['PBE_TiO2_Bulk']-0.05,\
     eov_bulk_lim['PBE_TiO2_Bulk']-0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs.plot([-2,130], np.array([eov_bulk_lim['PBE_TiO2_Bulk'],eov_bulk_lim['PBE_TiO2_Bulk']]),'k-',linewidth=1,label='Bulk limit')
axs.plot(RDF_sizes, RDF_eov, 'o',markersize=4,markeredgewidth=0.0,color=color_dict['blue'], label='RDF approach')
axs.plot(Stoic_sizes, Stoic_eov, 'o',markersize=4,markeredgewidth=0.0,linewidth=0,color=color_dict['red'], label='Stoichiometric')
axs.plot([3]*6, Ti3_configurations_eov, 'o',markersize=4,markeredgewidth=0.0,color=color_dict['red'])
axs.plot([18],SCC_eov,'o',markersize=4,markeredgewidth=0.0,color=color_dict['green'],label='SCC (18 Ti ion)')


axs.set_xlim([0,45])
axs.set_ylim([4.8,6.2])
axs.set_ylabel(r'$E_\textrm{Ov}$ (eV)')
axs.set_xlabel(r'Quantum cluster size (\# of Ti ions)')
axs.legend(fontsize=8,fancybox=False,loc='lower right')
axs.xaxis.set_label_coords(0.4, -0.15)
axs.set_yticks([4.8,5.0,5.2,5.4,5.6,5.8,6.0,6.2])
axs.xaxis.set_minor_locator(MultipleLocator(2))


plt.savefig('03-Figures/Fig_02.png')


In [ ]:

RDF_sizes = np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_perfect_TZVPP')[:,0]
RDF_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/RDF/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

Stoic_sizes = np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_perfect_TZVPP')[:,0]
Stoic_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

Ti3_configurations_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_Ti3_configurations_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/Stoic/energies_Ti3_configurations_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

SCC_eov = (np.loadtxt('02-Simulation_Data/Fig_02/Calculations/SCC/energies_defect_TZVPP')[1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_02/Calculations/SCC/energies_perfect_TZVPP')[1])*Hartree - O2_bind_ene['PBE']/2

fig, axs = plt.subplots(figsize=(2.23,2.5),dpi=1200, constrained_layout=True)

axs.fill_between([-10,150],[eov_bulk_lim['PBE_TiO2_Bulk']+0.05, eov_bulk_lim['PBE_TiO2_Bulk']+0.05],[eov_bulk_lim['PBE_TiO2_Bulk']-0.05,\
     eov_bulk_lim['PBE_TiO2_Bulk']-0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs.plot([-2,130], np.array([eov_bulk_lim['PBE_TiO2_Bulk'],eov_bulk_lim['PBE_TiO2_Bulk']]),'k-',linewidth=1,label='Bulk limit')
axs.plot(RDF_sizes, RDF_eov, 'o',markersize=4,markeredgewidth=0.0,color=color_dict['blue'], label='RDF approach')
axs.plot(Stoic_sizes, Stoic_eov, 'o',markersize=4,markeredgewidth=0.0,linewidth=0,color=color_dict['red'], label='Stoichiometric')
axs.plot([3], Ti3_configurations_eov[0], 'o',markersize=4,markeredgewidth=0.0,color=color_dict['red'])
# axs.plot([18],SCC_eov,'o',markersize=4,markeredgewidth=0.0,color=color_dict['green'],label='SCC (18 Ti ion)')


axs.set_xlim([0,45])
axs.set_ylim([4.8,6.2])
axs.set_ylabel(r'$E_\textrm{Ov}$ (eV)')
axs.set_xlabel(r'Quantum cluster size (\# of Ti ions)')
axs.legend(fontsize=8,fancybox=False,loc='lower right')
axs.xaxis.set_label_coords(0.4, -0.15)
axs.set_yticks([4.8,5.0,5.2,5.4,5.6,5.8,6.0,6.2])
axs.xaxis.set_minor_locator(MultipleLocator(2))


plt.savefig('03-Figures/Fig_02_RIG_talk.png')


### Figure 3 - EOv change with RDF cluster size for all 4 systems

In [15]:
RDF_MgO_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,0]
RDF_MgO_bulk_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_bulk_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_bulk = np.average(RDF_MgO_bulk_eov_TZVPP - RDF_MgO_bulk_eov_SVP[2:4])
RDF_MgO_bulk_eov = RDF_MgO_bulk_eov_SVP + SVP_to_TZVPP_corr_MgO_bulk

RDF_MgO_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,0]
RDF_MgO_surf_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_surf_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_surf = np.average(RDF_MgO_surf_eov_TZVPP - RDF_MgO_surf_eov_SVP[3:6])
RDF_MgO_surf_eov = RDF_MgO_surf_eov_SVP + SVP_to_TZVPP_corr_MgO_surf

RDF_TiO2_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,0]
RDF_TiO2_bulk_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

RDF_TiO2_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,0]
RDF_TiO2_surf_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE0_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE0']/2


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(6.69,5.2), sharey='row',dpi=1200)


axs[0,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,0].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,0].plot(RDF_MgO_bulk_sizes,RDF_MgO_bulk_eov - eov_bulk_lim['PBE_MgO_Bulk'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,0].plot(RDF_MgO_bulk_sizes[2],(RDF_MgO_bulk_eov)[2] - eov_bulk_lim['PBE_MgO_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,0].set_title(r'\textbf{MgO bulk}')
axs[0,0].text(0.05,0.85,'(a)', transform=axs[0,0].transAxes,fontsize=11)
axs[0,0].legend(fancybox=False,frameon=False,fontsize=8,loc='lower right')


axs[0,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,1].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,1].plot(RDF_MgO_surf_sizes,RDF_MgO_surf_eov - eov_bulk_lim['PBE_MgO_Surface'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,1].plot(RDF_MgO_surf_sizes[3],RDF_MgO_surf_eov[3] - eov_bulk_lim['PBE_MgO_Surface'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,1].set_title(r'\textbf{MgO surface}')
axs[0,1].text(0.05,0.85,'(b)', transform=axs[0,1].transAxes,fontsize=11)

axs[1,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,0].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,0].plot(RDF_TiO2_bulk_sizes,RDF_TiO2_bulk_eov - eov_bulk_lim['PBE_TiO2_Bulk'],'x--',color=color_dict['blue'],label='RDF approach')
axs[1,0].plot(RDF_TiO2_bulk_sizes[8],RDF_TiO2_bulk_eov[8] - eov_bulk_lim['PBE_TiO2_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[1,0].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,0].set_title(r'\textbf{TiO\textsubscript{2} bulk}')
axs[1,0].text(0.05,0.85,'(c)', transform=axs[1,0].transAxes,fontsize=11)
axs[1,0].set_ylabel(r'$\quad$ ')
fig.text(0.02, 0.5, 'Deviation from bulk limit (eV)', va='center', rotation='vertical')


axs[1,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,1].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,1].plot(RDF_TiO2_surf_sizes[:16], RDF_TiO2_surf_eov - eov_bulk_lim['PBE0_TiO2_Surface_2x6'] ,'x--',color=color_dict['blue'])
axs[1,1].plot(RDF_TiO2_surf_sizes[7], RDF_TiO2_surf_eov[7] - eov_bulk_lim['PBE0_TiO2_Surface_2x6'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'])
axs[1,1].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,1].set_title(r'\textbf{TiO\textsubscript{2} surface}')
axs[1,1].text(0.05,0.85,'(d)', transform=axs[1,1].transAxes,fontsize=11)

axs[0,0].set_xlim([0,120])
axs[0,0].set_ylim([-0.2, 0.3])
axs[0,0].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])
axs[0,0].xaxis.set_minor_locator(MultipleLocator(5))

axs[0,1].set_xlim([0,47])
axs[0,1].set_ylim([-0.2, 0.3])
axs[0,1].xaxis.set_minor_locator(MultipleLocator(1))
axs[0,1].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])

axs[1,0].set_xlim([0,47])
axs[1,0].set_ylim([-0.2,0.8])
axs[1,0].xaxis.set_minor_locator(MultipleLocator(1))

axs[1,1].set_xlim([0,47])
axs[1,1].set_ylim([-0.2,+0.8])
axs[1,1].xaxis.set_minor_locator(MultipleLocator(1))

fig.tight_layout()
plt.savefig('03-Figures/Fig_03.png')

In [21]:
RDF_MgO_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,0]
RDF_MgO_bulk_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_bulk_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_bulk = np.average(RDF_MgO_bulk_eov_TZVPP - RDF_MgO_bulk_eov_SVP[2:4])
RDF_MgO_bulk_eov = RDF_MgO_bulk_eov_SVP + SVP_to_TZVPP_corr_MgO_bulk

RDF_MgO_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,0]
RDF_MgO_surf_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_surf_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_surf = np.average(RDF_MgO_surf_eov_TZVPP - RDF_MgO_surf_eov_SVP[3:6])
RDF_MgO_surf_eov = RDF_MgO_surf_eov_SVP + SVP_to_TZVPP_corr_MgO_surf

RDF_TiO2_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,0]
RDF_TiO2_bulk_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

RDF_TiO2_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,0]
RDF_TiO2_surf_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE0_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE0']/2


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(6.69,5.2), sharey='row',dpi=1200)


axs[0,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,0].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,0].plot(RDF_MgO_bulk_sizes,RDF_MgO_bulk_eov - eov_bulk_lim['PBE_MgO_Bulk'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,0].plot(RDF_MgO_bulk_sizes[2],(RDF_MgO_bulk_eov)[2] - eov_bulk_lim['PBE_MgO_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,0].set_title(r'\textbf{MgO bulk}')
axs[0,0].text(0.05,0.85,'(a)', transform=axs[0,0].transAxes,fontsize=11)


axs[0,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,1].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,1].plot(RDF_MgO_surf_sizes,RDF_MgO_surf_eov - eov_bulk_lim['PBE_MgO_Surface'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,1].plot(RDF_MgO_surf_sizes[3],RDF_MgO_surf_eov[3] - eov_bulk_lim['PBE_MgO_Surface'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,1].set_title(r'\textbf{MgO surface}')
axs[0,1].text(0.05,0.85,'(b)', transform=axs[0,1].transAxes,fontsize=11)

axs[1,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,0].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
# axs[1,0].plot(RDF_TiO2_bulk_sizes,RDF_TiO2_bulk_eov - eov_bulk_lim['PBE_TiO2_Bulk'],'x--',color=color_dict['blue'],label='RDF approach')
# axs[1,0].plot(RDF_TiO2_bulk_sizes[8],RDF_TiO2_bulk_eov[8] - eov_bulk_lim['PBE_TiO2_Bulk'],marker='o',markeredgewidth=1.0,\
#      markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[1,0].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,0].set_title(r'\textbf{TiO\textsubscript{2} bulk}')
axs[1,0].text(0.05,0.85,'(c)', transform=axs[1,0].transAxes,fontsize=11)
axs[1,0].set_ylabel(r'$\quad$ ')
fig.text(0.02, 0.5, 'Deviation from bulk limit (eV)', va='center', rotation='vertical')
axs[1,0].legend(fancybox=False,frameon=False,fontsize=8,loc='upper right')


axs[1,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,1].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,1].plot(RDF_TiO2_surf_sizes[:16], RDF_TiO2_surf_eov - eov_bulk_lim['PBE0_TiO2_Surface_2x6'] ,'x--',color=color_dict['blue'])
axs[1,1].plot(RDF_TiO2_surf_sizes[7], RDF_TiO2_surf_eov[7] - eov_bulk_lim['PBE0_TiO2_Surface_2x6'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'])
axs[1,1].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,1].set_title(r'\textbf{TiO\textsubscript{2} surface}')
axs[1,1].text(0.05,0.85,'(d)', transform=axs[1,1].transAxes,fontsize=11)

axs[0,0].set_xlim([0,120])
axs[0,0].set_ylim([-0.2, 0.3])
axs[0,0].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])
axs[0,0].xaxis.set_minor_locator(MultipleLocator(5))

axs[0,1].set_xlim([0,47])
axs[0,1].set_ylim([-0.2, 0.3])
axs[0,1].xaxis.set_minor_locator(MultipleLocator(1))
axs[0,1].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])

axs[1,0].set_xlim([0,47])
axs[1,0].set_ylim([-0.2,0.8])
axs[1,0].xaxis.set_minor_locator(MultipleLocator(1))

axs[1,1].set_xlim([0,47])
axs[1,1].set_ylim([-0.2,+0.8])
axs[1,1].xaxis.set_minor_locator(MultipleLocator(1))

fig.tight_layout()
plt.savefig('03-Figures/Fig_03_RIG_talk_01.png')

No handles with labels found to put in legend.


In [22]:
RDF_MgO_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,0]
RDF_MgO_bulk_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_bulk_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_bulk = np.average(RDF_MgO_bulk_eov_TZVPP - RDF_MgO_bulk_eov_SVP[2:4])
RDF_MgO_bulk_eov = RDF_MgO_bulk_eov_SVP + SVP_to_TZVPP_corr_MgO_bulk

RDF_MgO_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,0]
RDF_MgO_surf_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_surf_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_surf = np.average(RDF_MgO_surf_eov_TZVPP - RDF_MgO_surf_eov_SVP[3:6])
RDF_MgO_surf_eov = RDF_MgO_surf_eov_SVP + SVP_to_TZVPP_corr_MgO_surf

RDF_TiO2_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,0]
RDF_TiO2_bulk_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

RDF_TiO2_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,0]
RDF_TiO2_surf_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE0_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE0']/2


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(6.69,5.2), sharey='row',dpi=1200)


axs[0,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,0].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,0].plot(RDF_MgO_bulk_sizes,RDF_MgO_bulk_eov - eov_bulk_lim['PBE_MgO_Bulk'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,0].plot(RDF_MgO_bulk_sizes[2],(RDF_MgO_bulk_eov)[2] - eov_bulk_lim['PBE_MgO_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,0].set_title(r'\textbf{MgO bulk}')
axs[0,0].text(0.05,0.85,'(a)', transform=axs[0,0].transAxes,fontsize=11)


axs[0,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,1].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,1].plot(RDF_MgO_surf_sizes,RDF_MgO_surf_eov - eov_bulk_lim['PBE_MgO_Surface'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,1].plot(RDF_MgO_surf_sizes[3],RDF_MgO_surf_eov[3] - eov_bulk_lim['PBE_MgO_Surface'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,1].set_title(r'\textbf{MgO surface}')
axs[0,1].text(0.05,0.85,'(b)', transform=axs[0,1].transAxes,fontsize=11)

axs[1,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,0].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,0].plot(RDF_TiO2_bulk_sizes,RDF_TiO2_bulk_eov - eov_bulk_lim['PBE_TiO2_Bulk'],'x--',color=color_dict['blue'],label='RDF approach')
# axs[1,0].plot(RDF_TiO2_bulk_sizes[8],RDF_TiO2_bulk_eov[8] - eov_bulk_lim['PBE_TiO2_Bulk'],marker='o',markeredgewidth=1.0,\
#      markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[1,0].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,0].set_title(r'\textbf{TiO\textsubscript{2} bulk}')
axs[1,0].text(0.05,0.85,'(c)', transform=axs[1,0].transAxes,fontsize=11)
axs[1,0].set_ylabel(r'$\quad$ ')
fig.text(0.02, 0.5, 'Deviation from bulk limit (eV)', va='center', rotation='vertical')
axs[1,0].legend(fancybox=False,frameon=False,fontsize=8,loc='upper right')


axs[1,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,1].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,1].plot(RDF_TiO2_surf_sizes[:16], RDF_TiO2_surf_eov - eov_bulk_lim['PBE0_TiO2_Surface_2x6'] ,'x--',color=color_dict['blue'])
axs[1,1].plot(RDF_TiO2_surf_sizes[7], RDF_TiO2_surf_eov[7] - eov_bulk_lim['PBE0_TiO2_Surface_2x6'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'])
axs[1,1].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,1].set_title(r'\textbf{TiO\textsubscript{2} surface}')
axs[1,1].text(0.05,0.85,'(d)', transform=axs[1,1].transAxes,fontsize=11)

axs[0,0].set_xlim([0,120])
axs[0,0].set_ylim([-0.2, 0.3])
axs[0,0].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])
axs[0,0].xaxis.set_minor_locator(MultipleLocator(5))

axs[0,1].set_xlim([0,47])
axs[0,1].set_ylim([-0.2, 0.3])
axs[0,1].xaxis.set_minor_locator(MultipleLocator(1))
axs[0,1].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])

axs[1,0].set_xlim([0,47])
axs[1,0].set_ylim([-0.2,0.8])
axs[1,0].xaxis.set_minor_locator(MultipleLocator(1))

axs[1,1].set_xlim([0,47])
axs[1,1].set_ylim([-0.2,+0.8])
axs[1,1].xaxis.set_minor_locator(MultipleLocator(1))

fig.tight_layout()
plt.savefig('03-Figures/Fig_03_RIG_talk_02.png')

In [ ]:
RDF_MgO_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,0]
RDF_MgO_bulk_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_bulk_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_bulk = np.average(RDF_MgO_bulk_eov_TZVPP - RDF_MgO_bulk_eov_SVP[2:4])
RDF_MgO_bulk_eov = RDF_MgO_bulk_eov_SVP + SVP_to_TZVPP_corr_MgO_bulk

RDF_MgO_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,0]
RDF_MgO_surf_eov_SVP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_SVP')[:,1] + O_ene_PBE_SVP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_SVP')[:,1])*Hartree - O2_bind_ene['PBE']/2
RDF_MgO_surf_eov_TZVPP = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
SVP_to_TZVPP_corr_MgO_surf = np.average(RDF_MgO_surf_eov_TZVPP - RDF_MgO_surf_eov_SVP[3:6])
RDF_MgO_surf_eov = RDF_MgO_surf_eov_SVP + SVP_to_TZVPP_corr_MgO_surf

RDF_TiO2_bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,0]
RDF_TiO2_bulk_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2

RDF_TiO2_surf_sizes = np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,0]
RDF_TiO2_surf_eov = (np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_defect_TZVPP')[:,1] + O_ene_PBE0_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_03/Calculations/TiO2_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE0']/2


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(6.69,5.2), sharey='row',dpi=1200)


axs[0,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,0].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,0].plot(RDF_MgO_bulk_sizes,RDF_MgO_bulk_eov - eov_bulk_lim['PBE_MgO_Bulk'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,0].plot(RDF_MgO_bulk_sizes[2],(RDF_MgO_bulk_eov)[2] - eov_bulk_lim['PBE_MgO_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,0].set_title(r'\textbf{MgO bulk}')
axs[0,0].text(0.05,0.85,'(a)', transform=axs[0,0].transAxes,fontsize=11)


axs[0,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[0,1].fill_between([-10,150],[+0.02, +0.02],[-0.02, -0.02],color='tab:gray',edgecolor=None,alpha=0.2)
axs[0,1].plot(RDF_MgO_surf_sizes,RDF_MgO_surf_eov - eov_bulk_lim['PBE_MgO_Surface'],'x--',color=color_dict['blue'],label='RDF Approach')
axs[0,1].plot(RDF_MgO_surf_sizes[3],RDF_MgO_surf_eov[3] - eov_bulk_lim['PBE_MgO_Surface'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[0,1].set_title(r'\textbf{MgO surface}')
axs[0,1].text(0.05,0.85,'(b)', transform=axs[0,1].transAxes,fontsize=11)

axs[1,0].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,0].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,0].plot(RDF_TiO2_bulk_sizes,RDF_TiO2_bulk_eov - eov_bulk_lim['PBE_TiO2_Bulk'],'x--',color=color_dict['blue'],label='RDF approach')
axs[1,0].plot(RDF_TiO2_bulk_sizes[8],RDF_TiO2_bulk_eov[8] - eov_bulk_lim['PBE_TiO2_Bulk'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'],label='Smallest converged cluster (SCC)')
axs[1,0].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,0].set_title(r'\textbf{TiO\textsubscript{2} bulk}')
axs[1,0].text(0.05,0.85,'(c)', transform=axs[1,0].transAxes,fontsize=11)
axs[1,0].set_ylabel(r'$\quad$ ')
fig.text(0.02, 0.5, 'Deviation from bulk limit (eV)', va='center', rotation='vertical')
axs[1,0].legend(fancybox=False,frameon=False,fontsize=8,loc='upper right')


axs[1,1].plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs[1,1].fill_between([-10,150],[+0.05, +0.05],[-0.05, -0.05],color='tab:gray',edgecolor=None,alpha=0.2)
axs[1,1].plot(RDF_TiO2_surf_sizes[:16], RDF_TiO2_surf_eov - eov_bulk_lim['PBE0_TiO2_Surface_2x6'] ,'x--',color=color_dict['blue'])
axs[1,1].plot(RDF_TiO2_surf_sizes[7], RDF_TiO2_surf_eov[7] - eov_bulk_lim['PBE0_TiO2_Surface_2x6'],marker='o',markeredgewidth=1.0,\
     markerfacecolor="None", linewidth=0,color=color_dict['red'])
axs[1,1].set_xlabel(r'Quantum cluster size (\# of Mg or Ti ions)')
axs[1,1].set_title(r'\textbf{TiO\textsubscript{2} surface}')
axs[1,1].text(0.05,0.85,'(d)', transform=axs[1,1].transAxes,fontsize=11)

axs[0,0].set_xlim([0,120])
axs[0,0].set_ylim([-0.2, 0.3])
axs[0,0].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])
axs[0,0].xaxis.set_minor_locator(MultipleLocator(5))

axs[0,1].set_xlim([0,47])
axs[0,1].set_ylim([-0.2, 0.3])
axs[0,1].xaxis.set_minor_locator(MultipleLocator(1))
axs[0,1].set_yticks([-0.2,-0.1,0.00,0.1,0.2,0.3])

axs[1,0].set_xlim([0,47])
axs[1,0].set_ylim([-0.2,0.8])
axs[1,0].xaxis.set_minor_locator(MultipleLocator(1))

axs[1,1].set_xlim([0,47])
axs[1,1].set_ylim([-0.2,+0.8])
axs[1,1].xaxis.set_minor_locator(MultipleLocator(1))

fig.tight_layout()
plt.savefig('03-Figures/Fig_03_RIG_talk_03.png')

### Figure 4 - Change in EOv with quantum cluster size for different levels of theory

In [16]:
eov_scc_updown_PBE = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/PBE/energies_defect_TZVPP')[:,1] + O_ene_PBE_TZVPP\
    -  np.loadtxt('02-Simulation_Data/Fig_04/Calculations/PBE/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE']/2
eov_scc_updown_PBE0 = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/PBE0/energies_defect_TZVPP')[:,1] + O_ene_PBE0_TZVPP\
    -  np.loadtxt('02-Simulation_Data/Fig_04/Calculations/PBE0/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['PBE0']/2
eov_scc_updown_MP2 = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/LMP2+HF/energies_MP2_defect_TZVPP')[:,1] + O_ene_LMP2_TZVPP \
    - np.loadtxt('02-Simulation_Data/Fig_04/Calculations/LMP2+HF/energies_MP2_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['LMP2']/2
eov_scc_updown_HF = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/LMP2+HF/energies_HF_defect_TZVPP')[:,1] + O_ene_HF_TZVPP \
     - np.loadtxt('02-Simulation_Data/Fig_04/Calculations/LMP2+HF/energies_HF_perfect_TZVPP')[:,1])*Hartree -  O2_bind_ene['PBE']/2 # Not important to have correct binding energy

fig, axs_10_inset = plt.subplots(figsize=(3.37,2.5), dpi=1200,constrained_layout=True)

cluster_sizes = np.array([14,16,18,22,26,30])


axs_10_inset.plot([-10,150],[0.0,0.0],'-k',linewidth=1)
axs_10_inset.fill_between([-10,150],[+0.03, +0.03],[-0.03, -0.03],color='tab:gray',edgecolor=None,alpha=0.2)
axs_10_inset.plot(cluster_sizes,(eov_scc_updown_PBE0- eov_scc_updown_PBE0[2]), 'x--',color=color_dict['red'],label='PBE0',linewidth=1)
axs_10_inset.plot(cluster_sizes,(eov_scc_updown_HF- eov_scc_updown_HF[2]), 'x--',color=color_dict['orange'],label='HF',linewidth=1)
axs_10_inset.plot(cluster_sizes,(eov_scc_updown_PBE- eov_scc_updown_PBE[2]), 'x--',color=color_dict['blue'],label='PBE',linewidth=1)
axs_10_inset.plot(cluster_sizes,(eov_scc_updown_MP2- eov_scc_updown_MP2[2]), 'x--',color=color_dict['green'],label='LMP2',linewidth=1)

axs_10_inset.set_xlim([13,31])
axs_10_inset.set_ylim([-0.05,0.20])
axs_10_inset.legend(fontsize=8,frameon=False,fancybox=False,loc='upper right')
axs_10_inset.set_ylabel('Deviation from SCC (eV)')
axs_10_inset.set_xticks(cluster_sizes)
axs_10_inset.set_yticks([-0.05,0.00,0.05,0.10,0.15,0.20])
axs_10_inset.set_xlabel(r'Quantum cluster size (\# of Ti ions)')

plt.savefig('03-Figures/Fig_04.png')


In [34]:
# For MgO Surface
eov_scc_updown_MgO_Surface_sizes = np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Surface/energies_defect_TZVPP')[:,0]

eov_scc_updown_MgO_Surface_LMP2 = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Surface/energies_defect_TZVPP')[:,2] + O_ene_LMP2_TZVPP\
    -  np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,2])*Hartree - O2_bind_ene['LMP2']/2

eov_scc_updown_MgO_Surface_LCCSDT = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Surface/energies_defect_TZVPP')[:,1] + O_ene_CCSDT_TZVPP\
    -  np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Surface/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['LNO-CCSD(T)']/2

for i in range(len(eov_scc_updown_MgO_Surface_sizes)):
    print('{0:2}  {1:.2f}'.format(int(eov_scc_updown_MgO_Surface_sizes[i]),eov_scc_updown_MgO_Surface_LMP2[i]))


 4  7.14
 5  7.05
 9  6.98
17  6.93
21  6.88
25  6.85
29  6.84
33  6.87
41  6.88
42  6.88


In [33]:
# For MgO Bulk

eov_scc_updown_MgO_Bulk_sizes = np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,0]

eov_scc_updown_MgO_Bulk_LMP2 = (np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Bulk/energies_defect_TZVPP')[:,1] + O_ene_LMP2_TZVPP\
    -  np.loadtxt('02-Simulation_Data/Fig_04/Calculations/MgO_Bulk/energies_perfect_TZVPP')[:,1])*Hartree - O2_bind_ene['LMP2']/2

for i in range(len(eov_scc_updown_MgO_Bulk_sizes)):
    print('{0:2}  {1:.2f}'.format(int(eov_scc_updown_MgO_Bulk_sizes[i]),eov_scc_updown_MgO_Bulk_LMP2[i]))

 6  7.68
14  7.48
38  7.33
68  7.31


### Table S6 - Basis set and frozen core correction for cWFT calculations including double hybrid B2PLYP

In [17]:
from cluster_scripts import *

eov_correction = dict.fromkeys(['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface'])
for i in eov_correction:
        eov_correction[i] = {'LB2PLYP':0.0, 'LMP2':0.0, 'LNO-CCSD':0.0, 'LNO-CCSD(T)':0.0}

capitalize_dict = {'lmp2': 'LMP2', 'lccsd':'LNO-CCSD','lccsdt':'LNO-CCSD(T)'}

# MgO Surface
for i in ['lmp2','lccsd','lccsdt']:
        eov_correction['MgO Surface'][capitalize_dict[i]] = get_corr_mrcc('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/MgO_Surface/CCSDT/4',i)[-1]
eov_correction['MgO Surface']['LB2PLYP'] = get_corr_mrcc_b2plyp('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/MgO_Surface/B2PLYP/4')[2]

# MgO Bulk
for i in ['lmp2','lccsd','lccsdt']:
        eov_correction['MgO Bulk'][capitalize_dict[i]] = get_corr_mrcc('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/MgO_Bulk/CCSDT/1',i)[-1]
eov_correction['MgO Bulk']['LB2PLYP'] = get_corr_mrcc_b2plyp('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/MgO_Bulk/B2PLYP/1')[2]

# TiO2 Bulk
for i in ['lmp2','lccsd','lccsdt']:
        method_correction = 0
        for j in ['2','3']:
                method_correction += get_corr_mrcc('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/TiO2_Bulk/CCSDT/{0}'.format(j),i)[-1]/2
        eov_correction['TiO2 Bulk'][capitalize_dict[i]] = method_correction

method_correction = 0
for j in ['2']:
        method_correction += get_corr_mrcc_b2plyp('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/TiO2_Bulk/B2PLYP/{0}'.format(j))[2]
eov_correction['TiO2 Bulk']['LB2PLYP'] = method_correction

# TiO2 Surface

for i in ['lmp2','lccsd','lccsdt']:
        method_correction = 0
        if i == 'lmp2':
                for j in ['1','2']:
                        method_correction += get_corr_TiO2_surf('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/TiO2_Surface/MP2/{0}'.format(j),i)[-1]/2
        else:
                for j in ['1','2']:
                        method_correction += get_corr_TiO2_surf('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/TiO2_Surface/CCSDT/{0}'.format(j),i)[-1]/2
        eov_correction['TiO2 Surface'][capitalize_dict[i]] = method_correction

method_correction = 0
for j in ['1','2']:
        method_correction += get_corr_TiO2_Surf_b2plyp('02-Simulation_Data/Table_S5-Basis_Set+Frozen_Core_Corrections/Calculations/TiO2_Surface/B2PLYP/{0}'.format(j))[2]/2

eov_correction['TiO2 Surface']['LB2PLYP'] = method_correction

df = pd.DataFrame(eov_correction)
rounded_df = df.round(decimals=2)
print(rounded_df)


             MgO Bulk  MgO Surface  TiO2 Bulk  TiO2 Surface
LB2PLYP          0.00         0.05       0.01         -0.04
LMP2             0.04         0.13      -0.10         -0.21
LNO-CCSD         0.06         0.14       0.11          0.13
LNO-CCSD(T)      0.06         0.16       0.13          0.11


### Figure 5 and Table 1 - Final DFT and cWFT EOv values and DFT XC functional errors

In [23]:
from cluster_scripts import *

eov_final = dict.fromkeys(['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface'])

all_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP','LMP2','LNO-CCSD','LNO-CCSD(T)']
mrcc_XC_functionals = ['BLYP','BP86','PBE','SCAN','TPSS','B3LYP','PBE0','HSE06']

for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:
        eov_final[i] = {i : 0.0 for i in all_methods}

# Computing the EOv values for the DFT functionals first
for i in mrcc_XC_functionals:
        eov_final['MgO Bulk'][i] = get_dft_vac_energy("02-Simulation_Data/Fig_05+Table_01/Calculations/MgO_Bulk/{0}".format(i)) - O2_bind_ene[i]/2
        eov_final['MgO Surface'][i] = get_dft_vac_energy("02-Simulation_Data/Fig_05+Table_01/Calculations/MgO_Surface/{0}".format(i)) - O2_bind_ene[i]/2
        eov_final['TiO2 Bulk'][i] = get_dft_vac_energy("02-Simulation_Data/Fig_05+Table_01/Calculations/TiO2_Bulk/{0}".format(i)) - O2_bind_ene[i]/2
        eov_final['TiO2 Surface'][i] = get_dft_vac_energy("02-Simulation_Data/Fig_05+Table_01/Calculations/TiO2_Surface/{0}".format(i)) - O2_bind_ene[i]/2

# Fixing issue with semilocal functionals not working in MRCC for TiO2 Surface
for i in  ['BLYP','BP86','PBE','SCAN','TPSS']:
    eov_final['TiO2 Surface'][i] = get_dft_vac_energy_orca("02-Simulation_Data/Fig_05+Table_01/Calculations/TiO2_Surface/{0}".format(i)) - O2_bind_ene[i]/2

# Calculations were very slow with wB97X calculations in MRCC, so they were performed in ORCA
eov_final['MgO Surface']['wB97X'] = get_dft_vac_energy_orca("02-Simulation_Data/Fig_05+Table_01/Calculations/MgO_Surface/wB97X".format(i)) - O2_bind_ene['wB97X']/2
eov_final['MgO Bulk']['wB97X'] = get_dft_vac_energy_orca_mgo_bulk("02-Simulation_Data/Fig_05+Table_01/Calculations/MgO_Bulk/wB97X".format(i)) - O2_bind_ene['wB97X']/2 # TZVPP calculation used as SCF convergence issues for QZVPP
eov_final['TiO2 Surface']['wB97X'] = get_dft_vac_energy_orca("02-Simulation_Data/Fig_05+Table_01/Calculations/TiO2_Surface/wB97X".format(i)) - O2_bind_ene['wB97X']/2
eov_final['TiO2 Bulk']['wB97X'] = get_dft_vac_energy_orca("02-Simulation_Data/Fig_05+Table_01/Calculations/TiO2_Bulk/wB97X".format(i)) - O2_bind_ene['wB97X']/2

# Now calculating B2PLYP and cWFT methods using CBS(TZVPP/QZVPP) + the basis set and frozen core correction calculated in cell above

for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:  
    ene_vac_hf = []
    ene_vac_mp2 = []
    nospace_name = '{0}_{1}'.format(i.split()[0],i.split()[1])
    for j in ['TZ','QZ']:
        ene_perfect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/perfect/mrcc.out'.format(nospace_name,j),typ='hf')
        ene_defect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/defect/mrcc.out'.format(nospace_name,j),typ='hf')
        ene_O = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/O/mrcc.out'.format(nospace_name,j),typ='hf')
        ene_vac_hf += [(ene_defect + ene_O - ene_perfect)*Hartree]
        ene_perfect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/perfect/mrcc.out'.format(nospace_name,j),typ='B2PLYP')
        ene_defect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/defect/mrcc.out'.format(nospace_name,j),typ='B2PLYP')
        ene_O = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/B2PLYP_{1}/O/mrcc.out'.format(nospace_name,j),typ='B2PLYP')
        ene_vac_mp2 += [(ene_defect + ene_O - ene_perfect)*Hartree]
    eov_final[i]['LB2PLYP'] = extrapolate.get_cbs(ene_vac_hf[0],ene_vac_mp2[0],ene_vac_hf[1],ene_vac_mp2[1],X=3,Y=4,family='def2',output=False)[2] - O2_bind_ene['LB2PLYP']/2 \
         + eov_correction[i]['LB2PLYP']

capitalize_dict = {'lmp2': 'LMP2', 'lccsd':'LNO-CCSD','lccsdt':'LNO-CCSD(T)'}

# MgO Surface
for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:
    nospace_name = '{0}_{1}'.format(i.split()[0],i.split()[1])
    for k in ['lmp2','lccsd','lccsdt']:  
        ene_vac_hf = []
        ene_vac_cwft = []
        for j in ['TZ','QZ']:    
            ene_perfect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/perfect/mrcc.out'.format(nospace_name,j),typ='hf')
            ene_defect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/defect/mrcc.out'.format(nospace_name,j),typ='hf')
            ene_O = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/O/mrcc.out'.format(nospace_name,j),typ='hf')
            ene_vac_hf += [(ene_defect + ene_O - ene_perfect)*Hartree]
            ene_perfect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/perfect/mrcc.out'.format(nospace_name,j),typ=k)
            ene_defect = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/defect/mrcc.out'.format(nospace_name,j),typ=k)
            ene_O = find_energy('02-Simulation_Data/Fig_05+Table_01/Calculations/{0}/CCSDT_{1}/O/mrcc.out'.format(nospace_name,j),typ=k[1:])
            ene_vac_cwft += [(ene_defect + ene_O - ene_perfect)*Hartree]

        eov_final[i][capitalize_dict[k]] = extrapolate.get_cbs(ene_vac_hf[0],ene_vac_cwft[0],ene_vac_hf[1],ene_vac_cwft[1],X=3,Y=4,shift=0.0,family='def2',output=False)[2] - O2_bind_ene[capitalize_dict[k]]/2 \
             + eov_correction[i][capitalize_dict[k]]

# Now obtaining the errors of the DFT methods
eov_error_final = dict.fromkeys(['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface'])

all_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP','LMP2','LNO-CCSD','LNO-CCSD(T)']
dft_error_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP']

dft_mae_errors = dict.fromkeys(all_methods)
for i in all_methods:
    total_abs_error = 0
    for j in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:
        total_abs_error += abs(eov_final[j][i] - eov_final[j]['LNO-CCSD(T)'])
    dft_mae_errors[i] = total_abs_error/4
        

for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface','MAE']:
    if i == 'MAE':
        eov_error_final[i] = {i1 : 0.0 for i1 in all_methods + ['MAE']}
        total_abs_error = 0
        for j in all_methods:
            eov_error_final[i][j] = dft_mae_errors[j]
        eov_error_final[i]['MAE'] = 0.0
    else:        
        eov_error_final[i] = {i1 : 0.0 for i1 in all_methods + ['MAE']}
        total_abs_error = 0
        for j in all_methods:
            error = eov_final[i][j] - eov_final[i]['LNO-CCSD(T)']
            if j in dft_error_methods:
                total_abs_error += abs(error)
            eov_error_final[i][j] = error
        eov_error_final[i]['MAE'] = total_abs_error/len(dft_error_methods)


# EOv values using the experimental binding energy instead
eov_final_expt_bind = dict.fromkeys(['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface'])

all_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP','LMP2','LNO-CCSD','LNO-CCSD(T)']

for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:
    eov_final_expt_bind[i] = {i : 0.0 for i in all_methods}
    for j in all_methods:
        eov_final_expt_bind[i][j] = eov_final[i][j] + O2_bind_ene[j]/2 - 5.22/2

eov_error_final_expt_bind = dict.fromkeys(['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface'])

all_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP','LMP2','LNO-CCSD','LNO-CCSD(T)']
dft_error_methods = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP']

dft_mae_errors_expt_bind = dict.fromkeys(all_methods)
for i in all_methods:
    total_abs_error = 0
    for j in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface']:
        total_abs_error += abs(eov_final_expt_bind[j][i] - eov_final_expt_bind[j]['LNO-CCSD(T)'])
    dft_mae_errors_expt_bind[i] = total_abs_error/4
        

for i in ['MgO Bulk','MgO Surface','TiO2 Bulk','TiO2 Surface','MAE']:
    if i == 'MAE':
        eov_error_final_expt_bind[i] = {i1 : 0.0 for i1 in all_methods + ['MAE']}
        total_abs_error = 0
        for j in all_methods:
            eov_error_final_expt_bind[i][j] = dft_mae_errors_expt_bind[j]
        eov_error_final_expt_bind[i]['MAE'] = 0.0
    else:        
        eov_error_final_expt_bind[i] = {i1 : 0.0 for i1 in all_methods + ['MAE']}
        total_abs_error = 0
        for j in all_methods:
            error = eov_final_expt_bind[i][j] - eov_final_expt_bind[i]['LNO-CCSD(T)']
            if j in dft_error_methods:
                total_abs_error += abs(error)
            eov_error_final_expt_bind[i][j] = error
        eov_error_final_expt_bind[i]['MAE'] = total_abs_error/len(dft_error_methods)


from matplotlib.lines import Line2D

fig, axs = plt.subplots(figsize=(3.37,3.2),dpi=1200, sharex = True,tight_layout=True)

axs.plot([-1, 20],[0.0,0.0],'k-',linewidth=1)
axs.fill_between([-10,150],[+0.20, +0.20],[-0.20, -0.20],color='tab:gray',edgecolor=None,alpha=0.2)

eov_method_sorted_list = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP','wB97X','LB2PLYP','LMP2','LNO-CCSD']
eov_method_sorted_list_val = [1,2,3,5,6,8,9,10,11,13,15,16]

mgo_surf_errors_sorted_list = []
mgo_bulk_errors_sorted_list = []
tio2_surf_errors_sorted_list = []
tio2_bulk_errors_sorted_list = []

for i in eov_method_sorted_list:
    mgo_surf_errors_sorted_list += [eov_error_final['MgO Surface'][i]]
    mgo_bulk_errors_sorted_list += [eov_error_final['MgO Bulk'][i]]
    tio2_surf_errors_sorted_list += [eov_error_final['TiO2 Surface'][i]]
    tio2_bulk_errors_sorted_list += [eov_error_final['TiO2 Bulk'][i]]


axs.plot(eov_method_sorted_list_val,mgo_surf_errors_sorted_list, marker='x',markeredgewidth=1.0, markerfacecolor="None",color=color_dict['red'], linewidth=0,label='MgO Surface',alpha=0.7)
axs.plot(eov_method_sorted_list_val,mgo_bulk_errors_sorted_list, marker='x', markeredgewidth=1.0, markerfacecolor="None",color=color_dict['blue'], linewidth=0, label='MgO Bulk',alpha=0.7)
axs.plot(eov_method_sorted_list_val,tio2_surf_errors_sorted_list, marker='o', markeredgewidth=1.0, markerfacecolor="None",color=color_dict['red'], linewidth=0, label=r'TiO\textsubscript{2} Surface',alpha=0.7)
axs.plot(eov_method_sorted_list_val,tio2_bulk_errors_sorted_list, marker='o', markeredgewidth=1.0, markerfacecolor="None",color=color_dict['blue'], linewidth=0, label=r'TiO\textsubscript{2} Bulk',alpha=0.7)

axs.set_xticks(eov_method_sorted_list_val)
eov_method_sorted_list1 = ['PBE','BP86','BLYP','TPSS','SCAN','PBE0','HSE06','B3LYP',r'$\omega$B97X','LB2PLYP','LMP2','LNO-CCSD']

axs.set_xticklabels(eov_method_sorted_list1,rotation=90)
axs.set_ylabel('Deviation from LNO-CCSD(T) (eV)')

for i in [0,4,7,12,14,17]:
    line = Line2D([i,i], [-1.5,-3.0], lw=1, color='k')
    line.set_clip_on(False)
    axs.add_line(line)

plt.figtext(0.295, 0.01, "GGA", ha="center", fontsize=9)
plt.figtext(0.455, 0.01, "mGGA", ha="center", fontsize=9)
plt.figtext(0.627, 0.01, "hybrid", ha="center", fontsize=9)
plt.figtext(0.785, 0.01, "DH", ha="center", fontsize=9)
plt.figtext(0.89, 0.01, "cWFT", ha="center", fontsize=9)
# cklabels(x)

axs.legend(fancybox=False,ncol=2,loc='lower right',fontsize=8,handletextpad=0.1)
axs.set_ylim([-1.5,0.5])
axs.set_xlim([0,17])

plt.savefig('03-Figures/Fig_05.png')



#### Final EOv values using the corresponding method's binding energy



In [19]:
df1 = pd.DataFrame(eov_final)
rounded_df1 = df1.round(decimals=2)
print(rounded_df1)
df1.to_csv('final_eov.csv')

             MgO Bulk  MgO Surface  TiO2 Bulk  TiO2 Surface
PBE              6.66         6.15       5.67          5.00
BP86             6.85         6.31       5.71          5.06
BLYP             7.11         6.54       5.66          4.91
TPSS             6.87         6.34       5.86          5.15
SCAN             7.43         6.74       6.27          5.43
PBE0             6.99         6.39       6.07          5.41
HSE06            7.06         6.46       6.08          5.40
B3LYP            7.41         6.76       6.05          5.32
wB97X            7.34         6.79       6.28          5.49
LB2PLYP          7.58         7.01       6.15          5.52
LMP2             7.84         7.41       6.44          5.52
LNO-CCSD         7.95         7.24       6.76          5.63
LNO-CCSD(T)      7.68         7.18       6.43          5.58


#### Errors in the final EOv values (with method binding energy)

In [20]:
df2 = pd.DataFrame(eov_error_final)
rounded_df2 = df2.round(decimals=2)
print(rounded_df2)
df2.to_csv('final_eov_error.csv')

             MgO Bulk  MgO Surface  TiO2 Bulk  TiO2 Surface   MAE
PBE             -1.02        -1.03      -0.76         -0.58  0.85
BP86            -0.83        -0.87      -0.72         -0.53  0.74
BLYP            -0.56        -0.64      -0.77         -0.67  0.66
TPSS            -0.81        -0.85      -0.57         -0.44  0.67
SCAN            -0.25        -0.44      -0.16         -0.15  0.25
PBE0            -0.69        -0.79      -0.36         -0.18  0.51
HSE06           -0.61        -0.73      -0.35         -0.18  0.47
B3LYP           -0.27        -0.42      -0.38         -0.27  0.33
wB97X           -0.34        -0.39      -0.15         -0.09  0.25
LB2PLYP         -0.10        -0.17      -0.28         -0.07  0.15
LMP2             0.16         0.23       0.01         -0.06  0.12
LNO-CCSD         0.27         0.06       0.32          0.05  0.18
LNO-CCSD(T)      0.00         0.00       0.00          0.00  0.00
MAE              0.55         0.63       0.45          0.32  0.00


#### Final EOv values using the experimental binding energy

In [21]:
df3 = pd.DataFrame(eov_final_expt_bind)
rounded_df3 = df3.round(decimals=2)
print(rounded_df3)
df3.to_csv('final_eov_expt.csv')

             MgO Bulk  MgO Surface  TiO2 Bulk  TiO2 Surface
PBE              7.16         6.64       6.16          5.50
BP86             7.31         6.77       6.18          5.52
BLYP             7.43         6.86       5.98          5.23
TPSS             7.00         6.47       5.99          5.28
SCAN             7.57         6.89       6.42          5.58
PBE0             7.07         6.47       6.15          5.49
HSE06            7.12         6.51       6.14          5.45
B3LYP            7.46         6.81       6.10          5.37
wB97X            7.43         6.88       6.37          5.58
LB2PLYP          7.65         7.08       6.22          5.59
LMP2             8.06         7.63       6.65          5.74
LNO-CCSD         7.74         7.03       6.55          5.43
LNO-CCSD(T)      7.65         7.15       6.40          5.56


#### Errors in the final EOv values (with experimental binding energy)

In [22]:
df4 = pd.DataFrame(eov_error_final_expt_bind)
rounded_df4 = df4.round(decimals=2)
print(rounded_df4)
df4.to_csv('final_eov_error_expt.csv')

             MgO Bulk  MgO Surface  TiO2 Bulk  TiO2 Surface   MAE
PBE             -0.49        -0.51      -0.24         -0.05  0.32
BP86            -0.34        -0.38      -0.23         -0.04  0.25
BLYP            -0.22        -0.29      -0.43         -0.33  0.32
TPSS            -0.65        -0.68      -0.41         -0.28  0.50
SCAN            -0.08        -0.26       0.02          0.02  0.09
PBE0            -0.58        -0.68      -0.25         -0.07  0.40
HSE06           -0.53        -0.64      -0.27         -0.10  0.39
B3LYP           -0.19        -0.34      -0.30         -0.19  0.25
wB97X           -0.22        -0.27      -0.03          0.03  0.14
LB2PLYP         -0.00        -0.07      -0.18          0.03  0.07
LMP2             0.41         0.48       0.25          0.18  0.33
LNO-CCSD         0.09        -0.12       0.14         -0.13  0.12
LNO-CCSD(T)      0.00         0.00       0.00          0.00  0.00
MAE              0.33         0.41       0.24          0.11  0.00


### Something Else